In [1]:
import torch
import torch.nn as nn
import numpy as np
import random

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class LIMUBERTModel(nn.Module):
    def __init__(self):
        super(LIMUBERTModel, self).__init__()
        self.embed = nn.Linear(6, 72)
        encoder_layer = nn.TransformerEncoderLayer(d_model=72, nhead=8)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=6)
        self.fc = nn.Linear(72, 72)

    def forward(self, x):
        x = self.embed(x)
        x = self.transformer(x)
        x = self.fc(x)
        return x

    def get_embeddings(self, x):
        x = self.embed(x)
        return x
def run_inference(model, data):
    model.eval()

    with torch.no_grad():
        inputs = torch.tensor(data, dtype=torch.float32)
        
        # Realiza la inferencia
        outputs = model(inputs)
        
        return outputs

def extract_segment(data, start_index, seq_len=120):
    """
    Extracts a 6-second segment from the data starting from start_index.
    
    Parameters:
        data (np.array): The dataset array of shape (N, W, F).
        start_index (int): The starting index for the segment.
        seq_len (int): The sequence length (default is 120).
    
    Returns:
        np.array: A segment of shape (1, seq_len, F).
    """
    end_index = start_index + 1
    segment = data[start_index:end_index, :seq_len, :]
    return segment



---------------------


In [3]:
#set_seed(42)


# Load your data
data_path = 'dataset/hhar/data_20_120.npy'
data = np.load(data_path)

# Cargar el modelo
model = LIMUBERTModel()
model_path = 'saved/pretrain_base_hhar_20_120/hhar.pt'
state_dict = torch.load(model_path, map_location=torch.device('cpu'))

model.load_state_dict(state_dict, strict=False)
model.eval()

# Extract a 6-second segment starting from index 0
start_index = 0
segment = extract_segment(data, start_index)

## DUDO SI ESTOY PASANDOLE LOS DATOS CORRECTAMENTE
predictions = run_inference(model, segment)

## DESCONOZCO COMO INTERPRETAE ESTOS VALORES DEBERIA CORRER EL MODELO DE CLASIFICADOR?
print(predictions)


tensor([[[-0.2674,  0.1997, -1.3127,  ...,  0.4257, -0.0884, -0.1613],
         [-0.2669,  0.1988, -1.3129,  ...,  0.4243, -0.0886, -0.1612],
         [-0.2682,  0.2016, -1.3147,  ...,  0.4215, -0.0907, -0.1603],
         ...,
         [-0.2912,  0.2410, -1.3291,  ...,  0.3796, -0.1114, -0.1481],
         [-0.2915,  0.2413, -1.3307,  ...,  0.3753, -0.1126, -0.1477],
         [-0.2918,  0.2418, -1.3302,  ...,  0.3750, -0.1126, -0.1476]]])


In [41]:

segment = extract_segment(data, start_index,seq_len=120) # revisar funcion
segment.size

720

In [39]:
predictions = run_inference(model, segment)
print(predictions.size())
print(predictions)


torch.Size([1, 120, 72])
tensor([[[-0.2674,  0.1997, -1.3127,  ...,  0.4257, -0.0884, -0.1613],
         [-0.2669,  0.1988, -1.3129,  ...,  0.4243, -0.0886, -0.1612],
         [-0.2682,  0.2016, -1.3147,  ...,  0.4215, -0.0907, -0.1603],
         ...,
         [-0.2912,  0.2410, -1.3291,  ...,  0.3796, -0.1114, -0.1481],
         [-0.2915,  0.2413, -1.3307,  ...,  0.3753, -0.1126, -0.1477],
         [-0.2918,  0.2418, -1.3302,  ...,  0.3750, -0.1126, -0.1476]]])
